In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from Project_1_CRUD import AnimalDB

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalDB(username, password, 'nv-desktop-services.apporto.com', '31879', 'AAC', 'animal')

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################

app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                 style={'width': '200px', 'height': '200px', 'display': 'inline-block'}),
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        html.P('By: John Schatzl'),
    ], style={
        'textAlign': 'center',
        
    }),
    html.Hr(),
    html.Div([
        html.H2('Filter by Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain Wilderness'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster Individual'},
                {'label': 'Reset', 'value': 'Reset'} # Reset option.
            ],
            value='Reset' # Defaults to 'Reset' to show all of the data.
        )
    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         style_data_conditional=[ # Alternating row colors for readability.
                             {
                                 'if': {'row_index': 'odd'},
                                 'backgroundColor': 'rgb(248, 248, 248)'
                             }
                         ],
                         page_size=10, # Limits the rows per page.
                         sort_action='native', # Enables sorting.
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    query = {}
    if filter_type != 'Reset':
        if filter_type == 'Water Rescue':
            query['breed'] = {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}
        elif filter_type == 'Mountain Wilderness':
            query['breed'] = {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}
        elif filter_type == 'Disaster Individual':
            query['breed'] = {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}
        else:
            # Handling unexpected filter types.
            pass
    
    if 'breed' in query:
        breed_list = query['breed']['$in']
        filtered_df = df.query('breed in @breed_list')
    else:
        filtered_df = df
        
    if filtered_df.empty:
        return []
    else:
        return filtered_df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return dash.no_update
    
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Preferred Breeds')
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
def update_map(viewData, index):
    print("viewData: ", viewData)
    dff = pd.DataFrame.from_dict(viewData)
    
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    if row < len(dff) and not pd.isna(dff.iloc[row, 13]) and not pd.isna(dff.iloc[row, 14]):
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], 
                          children=[
                              dl.Tooltip(dff.iloc[row,4]),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(dff.iloc[row,9])
                              ])
                          ])
            ])
        ]
    else:
        return dash.no_update
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map_callback(viewData, index):
    return update_map(viewData, index)

# @app.callback(
#     Output('map-id', "children"),
#     [Input('datatable-id', "derived_virtual_data"),
#      Input('datatable-id', "derived_virtual_selected_rows")])
# def update_map(viewData, index):  
#     if viewData is None:
#         return
#     elif index is None:
#         return
    
#     dff = pd.DataFrame.from_dict(viewData)
#     # Because we only allow single row selection, the list can be converted to a row index here
#     if index is None:
#         row = 0
#     else: 
#         row = index[0]
        
#     # Austin TX is at [30.75,-97.48]
#     return [
#         dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
#             dl.TileLayer(id="base-layer-id"),
#             # Marker with tool tip and popup
#             # Column 13 and 14 define the grid-coordinates for the map
#             # Column 4 defines the breed for the animal
#             # Column 9 defines the name of the animal
#             dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
#                 dl.Tooltip(dff.iloc[row,4]),
#                 dl.Popup([
#                     html.H1("Animal Name"),
#                     html.P(dff.iloc[row,9])
#                 ])
#             ])
#         ])
#     ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:10182/
viewData:  None


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



viewData:  [{'rec_num': 9, 'age_upon_outcome': '3 years', 'animal_id': 'A720214', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Red/White', 'date_of_birth': '2013-02-04', 'datetime': '2016-02-11 12:41:00', 'monthyear': '2016-02-11T12:41:00', 'name': 'Blessing', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.3870648199411, 'location_long': -97.3684339731375, 'age_upon_outcome_in_weeks': 157.504067460317}, {'rec_num': 5, 'age_upon_outcome': '2 years', 'animal_id': 'A691584', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Tan/White', 'date_of_birth': '2012-11-06', 'datetime': '2015-05-30 13:48:00', 'monthyear': '2015-05-30T13:48:00', 'name': 'Luke', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.7104815618433, 'location_long': -97.562297435286, 'age_upon_outcome_in_weeks': 133.653571428571}, {'rec_num': 22, 'age_upon_outcom

viewData:  [{'rec_num': 304, 'age_upon_outcome': '13 years', 'animal_id': 'A358879', 'animal_type': 'Dog', 'breed': 'German Shepherd', 'color': 'Brown/Black', 'date_of_birth': '2002-08-13', 'datetime': '2015-09-15 16:22:00', 'monthyear': '2015-09-15T16:22:00', 'name': 'Simba', 'outcome_subtype': 'Suffering', 'outcome_type': 'Euthanasia', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.4499994838973, 'location_long': -97.6185491903329, 'age_upon_outcome_in_weeks': 683.097420634921}, {'rec_num': 400, 'age_upon_outcome': '11 months', 'animal_id': 'A681819', 'animal_type': 'Dog', 'breed': 'German Shepherd', 'color': 'Black', 'date_of_birth': '2013-07-15', 'datetime': '2014-07-02 16:40:00', 'monthyear': '2014-07-02T16:40:00', 'name': 'Ashes', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.502516926408, 'location_long': -97.5173436126027, 'age_upon_outcome_in_weeks': 50.3849206349206}, {'rec_num': 434, 'age_upon_outco

viewData:  [{'rec_num': 234, 'age_upon_outcome': '2 years', 'animal_id': 'A734683', 'animal_type': 'Dog', 'breed': 'Golden Retriever', 'color': 'Red', 'date_of_birth': '2014-09-11', 'datetime': '2016-09-15 11:24:00', 'monthyear': '2016-09-15T11:24:00', 'name': 'Sapphire', 'outcome_subtype': 'Partner', 'outcome_type': 'Transfer', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.4861149595305, 'location_long': -97.2691161890552, 'age_upon_outcome_in_weeks': 105.067857142857}, {'rec_num': 304, 'age_upon_outcome': '13 years', 'animal_id': 'A358879', 'animal_type': 'Dog', 'breed': 'German Shepherd', 'color': 'Brown/Black', 'date_of_birth': '2002-08-13', 'datetime': '2015-09-15 16:22:00', 'monthyear': '2015-09-15T16:22:00', 'name': 'Simba', 'outcome_subtype': 'Suffering', 'outcome_type': 'Euthanasia', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.4499994838973, 'location_long': -97.6185491903329, 'age_upon_outcome_in_weeks': 683.097420634921}, {'rec_num': 400, 'age_upon_outc

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

